# Install

In [ ]:
!pip install alpaca_trade_api -q
!pip install alpaca-py -q

# API CONNECT

In [1]:
import pandas as pd
from alpaca.trading.client import TradingClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical import StockHistoricalDataClient

from decouple import config

KEY_1 = config('KEY_1')
KEY_2 = config('KEY_2')

# Initialize the Alpaca TradingClient
trading_client = TradingClient(KEY_1,KEY_2,paper=True)

# Get all positions and create a Pandas dataframe
account_positions = trading_client.get_all_positions()
df_my_positions = pd.DataFrame.from_records(
  [dict(item) for item in account_positions])
# print(df_my_positions.info())

# Get account information
account = trading_client.get_account()
cash = float(account.cash)
print(f"\nCash i have : {cash}\n")



Cash i have : 799746.71



In [2]:
df_my_positions[['symbol','qty','market_value','cost_basis','unrealized_pl','unrealized_plpc','current_price','lastday_price','change_today']]

,symbol,qty,market_value,cost_basis,unrealized_pl,unrealized_plpc,current_price,lastday_price,change_today
0,AAPL,1,173.57,168.29,5.28,0.0313744132152831,173.57,173.57,0
1,BATUSD,997.8,231.68916,254.23944,-22.55028,-0.0886970172684458,0.2322,0.2422,-0.0412881915772089
2,BTCUSD,5.98679999,173750.8250857768,179664.62108448260816891,-5913.79599870580816891,-0.0329157513761432,29022.32,29643.96,-0.0209702077590174
3,DOGEUSD,998.7978,77.4068295,80.0524293474,-2.6455998474,-0.0330483393067187,0.0775,0.0798,-0.0288220551378446
4,ETHUSD,8.9648,17172.432992,17573.180684953764,-400.747692953764,-0.0228045053504108,1915.54,1990.81,-0.0378087311194941
5,GME,1,20.42,18.92,1.5,0.0792811839323467,20.42,20.42,0
6,LTCUSD,14.967,1257.97635,1331.840546017812,-73.864196017812,-0.0554602397701925,84.05,88.02,-0.0451033855941831
7,MSFT,2,621.3,608.11,13.19,0.0216901547417408,310.65,310.65,0
8,SPY,0.023,9.49049,9.551854,-0.061364,-0.006424302549013,412.63,412.63,0


In [2]:
# List of coins to trade
coins = [
  "AAVEUSD",
  "BATUSD",
  "BTCUSD",
  "BCHUSD",
  "LINKUSD",
  "DAIUSD",
  "DOGEUSD",
  "ETHUSD",
  "GRTUSD",
  "LTCUSD",
  "MKRUSD",
  "MATICUSD",
  "PAXGUSD",
  "SHIBUSD",
  "SOLUSD",
  "SUSHIUSD",
  "USDTUSD",
  "TRXUSD",
  "UNIUSD",
  "WBTCUSD",
  "YFIUSD",
]

coins_now = [
  "AAVE/USD",
  "BAT/USD",
  "BTC/USD",
  "BCH/USD",
  "LINK/USD",
  "DAI/USD",
  "DOGE/USD",
  "ETH/USD",
  "GRT/USD",
  "LTC/USD",
  "MKR/USD",
  "MATIC/USD",
  "PAXG/USD",
  "SHIB/USD",
  "SOL/USD",
  "SUSHI/USD",
  "USDT/USD",
  "TRX/USD",
  "UNI/USD",
  "WBTC/USD",
  "YFI/USD",
]



# Filter positions based on buying power
buying_power = account.buying_power
print(buying_power)

3201239.04196


In [3]:
from alpaca.data import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoLatestQuoteRequest

# no keys required.
crypto_client = CryptoHistoricalDataClient()

In [4]:
coin = "BTC/USD"
request_params = CryptoLatestQuoteRequest(symbol_or_symbols=coin)
latest_quote = crypto_client.get_crypto_latest_quote(request_params)
latest_quote[coin].ask_price

29048.389

In [ ]:
# from alpaca_trade_api.rest import REST, TimeFrame

# # Get the current price of BTCUSD

# coin_price = trading_client.get_asset("BTCUSD")
# coin_price

In [25]:
# import APIError
from alpaca_trade_api.rest import APIError,HTTPError

# Buying all coins equally
num_coins = len(coins)
initial_cash = float(trading_client.get_account().cash)
initial_alloc = 1.0 / num_coins
coin_positions = {}

for coin in coins_now:
    # Buy coin
    try:
        print(f"BUYING {coin}")
        #if position exists, skip
        
        
        request_params = CryptoLatestQuoteRequest(symbol_or_symbols=coin)
        latest_quote = crypto_client.get_crypto_latest_quote(request_params)
        coin_price = float(latest_quote[coin].ask_price)
        coin_qty = int((initial_cash * initial_alloc) / coin_price)
    except APIError as e:
        print(f"NO POSITION FOR {coin}")
        coin_price = 0.0
        coin_qty = int((initial_cash * initial_alloc) / coin_price)
        trading_client.submit_order(
            symbol=coin,
            qty=coin_qty,
            side="buy",
            type="market",
            time_in_force="gtc",
        )
        coin_positions[coin] = {"quantity": coin_qty, "price": coin_price}


BUYING AAVE/USD
BUYING BAT/USD
BUYING BTC/USD
BUYING BCH/USD
BUYING LINK/USD
BUYING DAI/USD
BUYING DOGE/USD
BUYING ETH/USD
BUYING GRT/USD
BUYING LTC/USD
BUYING MKR/USD
BUYING MATIC/USD
BUYING PAXG/USD
BUYING SHIB/USD
BUYING SOL/USD
BUYING SUSHI/USD
BUYING USDT/USD
BUYING TRX/USD


ZeroDivisionError: float division by zero

In [ ]:
import time

# Main loop to check coin prices
while True:
    # Wait for 5 seconds
    time.sleep(5)

    for coin, position in coin_positions.items():
        # Get current price
        current_price = float(trading_client.get_last_trade(coin).price)

        # If price has gone up 3% from initial timestamp, sell 1% position
        if current_price > position["price"] * 1.03:
            sell_qty = max(1, int(position["quantity"] * 0.01))
            trading_client.submit_order(
                symbol=coin,
                qty=sell_qty,
                side="sell",
                type="market",
                time_in_force="gtc",
            )
            position["quantity"] -= sell_qty
            time.sleep(120)
            sell_qty = max(1, int(position["quantity"] * 0.01))
            trading_client.submit_order(
                symbol=coin,
                qty=sell_qty,
                side="sell",
                type="market",
                time_in_force="gtc",
            )
            position["quantity"] -= sell_qty

        # If price has gone down 4% from initial timestamp, buy 1% position
        elif current_price < position["price"] * 0.96:
            buy_qty = max(1, int(initial_cash * 0.01 / current_price))
            trading_client.submit_order(
                symbol=coin,
                qty=buy_qty,
                side="buy",
                type="market",
                time_in_force="gtc",
            )
            position["quantity"] += buy_qty
            time.sleep(18000) # Wait for 5 hours
            current_price = float(trading_client.get_last_trade(coin).price)
            if current_price < position["price"] * 0.96:
                buy_qty = max(1, int(initial_cash * 0.01 / current_price))
                trading_client.submit_order(
                    symbol=coin,
                    qty=buy_qty,
                    side="buy",
                    type="market",
                    time_in_force="gtc",
                )
                position["quantity"] += buy

In [7]:
df_my_positions["market_value"] = pd.to_numeric(
  df_my_positions["market_value"])
# print(df_my_positions['market_value'])
buying_power = float(account.buying_power)

df_my_positions = df_my_positions[
  df_my_positions["market_value"] <= buying_power]

In [10]:
df_my_positions[['symbol','qty','market_value','cost_basis','unrealized_pl','unrealized_plpc','current_price','lastday_price','change_today']]

,symbol,qty,market_value,cost_basis,unrealized_pl,unrealized_plpc,current_price,lastday_price,change_today
0,AAPL,1,165.550000,168.29,-2.74,-0.0162814189791431,165.55,167.45,-0.0113466706479546
1,BATUSD,997.8,237.576180,254.23944,-16.66326,-0.065541601255887,0.2381,0.2433,-0.0213727907932594
2,BTCUSD,5.98679999,172775.575367,179664.62108448260816891,-6889.04571707680816891,-0.0383439192173367,28859.42,29132.6,-0.0093771239092975
3,DOGEUSD,998.7978,78.705267,80.0524293474,-1.3471627074,-0.0168285049983153,0.0788,0.0795,-0.0088050314465409
4,ETHUSD,8.9648,16840.556096,17573.180684953764,-732.624588953764,-0.0416899252382376,1878.52,1905,-0.0139002624671916
5,GME,1,19.320000,18.92,0.4,0.0211416490486258,19.32,18.9,0.0222222222222222
6,LTCUSD,14.967,1322.933130,1331.840546017812,-8.907416017812,-0.0066880498903905,88.39,88.56,-0.0019196025293586
7,MSFT,2,612.640000,608.11,4.53,0.0074493101577017,306.32,304.4,0.0063074901445466
8,SPY,0.023,9.323050,9.551854,-0.228804,-0.0239538837172344,405.35,408.02,-0.0065437968727023


In [8]:
# Loop through positions and add orders if necessary
for index, row in df_my_positions.iterrows():
  symbol = row["symbol"]
  market_value = row["market_value"]
  allocation = market_value / (market_value + cash)
  if allocation < 0.99:
    # If the position is 1% smaller than cash, buy
    quantity = int((buying_power - market_value) / float(row["current_price"]))

    #print("BUY quantity:", quantity)
    # trading_client.submit_order(
    #     symbol=symbol,
    #     qty=quantity,
    #     side="buy",
    #     type="market",
    #     time_in_force="gtc",
    # )
    print(f"Buying {quantity} shares of {symbol}")
  elif allocation > 1.01:
    # If the position is 1% higher than cash, sell
    quantity = int(market_value / row["current_price"])
    #print("SELL quantity:", quantity)
    # trading_client.submit_order(
    #     symbol=symbol,
    #     qty=quantity,
    #     side="sell",
    #     type="market",
    #     time_in_force="gtc",
    # )
    print(f"Selling {quantity} shares of {symbol}")

Buying 19335 shares of AAPL
Buying 13443717 shares of BATUSD
Buying 104 shares of BTCUSD
Buying 40623196 shares of DOGEUSD
Buying 1695 shares of ETHUSD
Buying 165691 shares of GME
Buying 36201 shares of LTCUSD
Buying 10448 shares of MSFT
Buying 7897 shares of SPY
